Based on https://github.com/eldar/pose-tensorflow/blob/6a8f5ee90ea444e12c15eecebf8448797c52bbf5/eval_pck.py

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
tf.keras.backend.set_learning_phase(0)

In [2]:
model = tf.keras.models.load_model('data/train/model.h5', compile=False)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
batch_size = 32
n_stack = 1
sigma = 1.0

num_images = 256
num_joints = 7

In [4]:
from Data_utils import DataGenerator, postprocessing

In [5]:
test_gen = DataGenerator('data/dataset/', 'test_landmarks.csv', 
                         inres=(128,128), outres=(32,32), nparts=7)

In [6]:
data = next(test_gen.generator(len(test_gen),n_stack,sigma=sigma))

In [7]:
pred = model.predict(data[0], batch_size=1)
pred = np.array([postprocessing(pred[i:i+1])[0] for i in range(len(pred))])

In [8]:
gt =np.array([postprocessing(data[1][0][i:i+1])[0] for i in range(len(data[1][0]))])

In [9]:
print(gt.shape)
print(pred.shape)

(256, 7, 2)
(256, 7, 2)


In [50]:
def calculate_PCKh(pred, gt, num_joints, num_images, pck_ratio_thresh=0.5, show_results=False):
    pred_joints = pred
    gt_joints = gt
    pck_thresh = np.zeros((num_images, 1))
    gt_present_joints = np.zeros((num_images, num_joints))
    
    def enclosing_rect(points):
        xs = points[:, 0]
        ys = points[:, 1]
        return np.array([np.amin(xs), np.amin(ys), np.amax(xs), np.amax(ys)])
    
    def rect_size(rect):
        return np.array([rect[2]-rect[0], rect[3]-rect[1]])
    
    for k in range(num_images):
        pred_, gt_ = pred[k], gt[k]
        gt_joint_ids = np.arange(0,num_joints)
        rect = enclosing_rect(gt_)
        pck_thresh[k] = pck_ratio_thresh*np.amax(rect_size(rect))
        gt_present_joints[k, gt_joint_ids] = 1
        
    dists = np.sqrt(np.sum((pred_joints - gt_joints)**2, axis=2))
    correct = dists <= pck_thresh
    
    num_all = np.sum(gt_present_joints, axis=0)

    num_correct = np.zeros((num_joints, ))
    for j_id in range(num_joints):
        num_correct[j_id] = np.sum(correct[gt_present_joints[:,j_id] == 1, j_id], axis=0)

    pck = num_correct/num_all*100.0
    
    if show_results:
        print(" & ".join(['left', 'right', 'center', 'lc', 'rc', 'up', 'down', 'total']))
        print(" & ".join([str(x) for x in pck] + [str(pck.mean())]))
        
    return pck

In [51]:
pck = calculate_PCKh(pred, gt, num_joints, num_images, pck_ratio_thresh=0.5, show_results=True)

left & right & center & lc & rc & up & down & total
68.75 & 68.75 & 99.609375 & 97.65625 & 99.609375 & 100.0 & 100.0 & 90.625
